**Homework 2:**
Top university rankings


# topuniversities.com section
Strategy for www.topuniversities.com website:
1. Analysis of web traffic after sending GET request shows that basic informations can be retrived just by getting json file from website.
2. We created DataFrame, from json, which contains basic information and urls to details page of every university.
3. Next step was to analyse detail page structure to obtain as much information as we can. This step make use of BeautifulSoup library.
4. In order to get details of each university we iterated through url and nid attributes of basic DataFrame and downloaded detail page together with json file (containing more information)
5. After downloading detail page we extracted details using BeautifulSoup and create DF. We did that with every university concatenating created DF.
6. At the end we merged (joined) both basic and details DF together on url attribute.
7. Cleaning the data was done in required places.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import difflib

In [2]:
# Get basic ranking using the same api as website does. Link comes from analysing web traffic after GET request to topunieversities.com website
top_universities_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508409283658'
r = requests.get(top_universities_url).json()

In [3]:
# Take only first 200 records and put them into DataFrame
top_unis = pd.DataFrame.from_dict(r['data'][:200])
top_unis = top_unis[['rank_display', 'title', 'score', 'cc', 'country', 
       'region', 'url', 'nid']]

In [4]:
details = pd.DataFrame()
for url, nid in zip(top_unis['url'].values, top_unis['nid'].values):
    #print(url)

    req = requests.get('https://www.topuniversities.com' + url)
    soup = BeautifulSoup(req.text, 'html.parser')
    req2 = requests.get('https://www.topuniversities.com/qs-profiles/classifications/' + nid).json()

    d = dict()
    d['url'] = url
    try:
        d['total faculty'] = int(soup.find("div", class_="total faculty").find("div", class_="number").string.strip().replace(",", ""))
        d['inter faculty'] = int(soup.find("div", class_="inter faculty").find("div", class_="number").string.strip().replace(",", ""))
        d['Number of students'] = int(soup.find("div", class_="total student").find("div", class_="number").string.strip().replace(",", ""))
        d['Postgraduate students [%]'] = int(soup.find("div", class_="total student").find("div", class_="post").span.string.strip().replace("%", ""))
        d['Undergraduate students [%]'] = int(soup.find("div", class_="total student").find("div", class_="grad").span.string.strip().replace("%", ""))
        d['Total international students'] = int(soup.find("div", class_="total inter").find("div", class_="number").string.strip().replace(",", ""))
        d['Postgraduate international students [%]'] = int(soup.find("div", class_="total inter").find("div", class_="post").span.string.strip().replace("%", ""))
        d['Undergraduate international students [%]'] = int(soup.find("div", class_="total inter").find("div", class_="grad").span.string.strip().replace("%", ""))

        #for s in soup.find("ul", class_="score").strings:
        #    print(s)

        #soup.find("div", id="wurc").find("ul", class_="score")

        score_type = [elem.string.strip() for elem in soup.find("ul", class_="score").find_all("div", class_="criteria")]
        score_value = [float(elem.string) for elem in soup.find("ul", class_="score").find_all("div", class_="text")]
        lis = list(zip(score_type, score_value))

        d.update(dict((type_, value_) for type_, value_ in lis))
        d.update(req2['classifications'])
        #temp = soup.find("ul", id = "uni-info").find_all("h4")

        #d['Age'] = temp[1].span.string
        #d['Status'] = temp[2].span.string
        #d['Research'] = temp[3].span.string
        #d['Focus'] = temp[4].span.string
        #for element in temp:
        #   d[element.contents[0].strip()] = element.span.string.strip()
    except:
        pass
    details = pd.concat([details, pd.DataFrame.from_dict([d])], ignore_index=True)
    
details.head()

,Academic Reputation,Citations per Faculty,Employer Reputation,Faculty Student,International Faculty,International Students,Number of students,Overall Score,Postgraduate international students [%],Postgraduate students [%],...,Undergraduate international students [%],Undergraduate students [%],age_value,data_status_value,inter faculty,research_intensity_value,size_value,subject_range_value,total faculty,url
0,100.0,99.9,100.0,100.0,100.0,96.1,11067.0,100.0,83.0,60.0,...,17.0,40.0,5,B,1679.0,VH,M,CO,2982.0,/universities/massachusetts-institute-technolo...
1,100.0,99.4,100.0,100.0,99.6,72.7,15878.0,98.7,83.0,56.0,...,17.0,44.0,5,B,2042.0,VH,L,FC,4285.0,/universities/stanford-university
2,100.0,99.9,100.0,98.3,96.5,75.2,22429.0,98.4,85.0,69.0,...,15.0,31.0,5,B,1311.0,VH,L,FC,4350.0,/universities/harvard-university
3,99.5,100.0,85.4,100.0,93.4,89.2,2255.0,97.7,87.0,56.0,...,13.0,44.0,5,B,350.0,VH,S,CO,953.0,/universities/california-institute-technology-...
4,100.0,78.3,100.0,100.0,97.4,97.7,18770.0,95.6,64.0,37.0,...,36.0,63.0,5,A,2278.0,VH,L,FC,5490.0,/universities/university-cambridge


In [5]:
# Merge of two DF on url column
Total = top_unis.merge(details, left_on='url', right_on='url', how='outer')
# Rename rank column and getting rid of "=" sign in rank column
Total.rename(columns={'rank_display': 'Rank TU'}, inplace=True)
Total['Rank TU'] = Total['Rank TU'].str.replace("=", "")

# timeshighereducation.com section
Strategy for www.timeshighereducation.com:
1. Similarly as in previous task we analysed web traffic (Postman app or Developer tools) after sending GET request and found an url to the json file with all needed informations. DataFrame was created using that json file.
2. Besides data cleaning and renaming or droping attributes, no futher actions were applied because of very handy json file structure.

In [6]:
times_rank_url = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r = requests.get(times_rank_url).json()

In [7]:
top_unis_times = pd.DataFrame.from_dict(r['data'][:200])

In [8]:
top_unis_times['stats_number_students'] = top_unis_times.stats_number_students.str.replace(",", "")

In [9]:
top_unis_times.columns

Index(['aliases', 'location', 'member_level', 'name', 'nid', 'rank',
       'rank_order', 'record_type', 'scores_citations',
       'scores_citations_rank', 'scores_industry_income',
       'scores_industry_income_rank', 'scores_international_outlook',
       'scores_international_outlook_rank', 'scores_overall',
       'scores_overall_rank', 'scores_research', 'scores_research_rank',
       'scores_teaching', 'scores_teaching_rank', 'stats_female_male_ratio',
       'stats_number_students', 'stats_pc_intl_students',
       'stats_student_staff_ratio', 'subjects_offered', 'url'],
      dtype='object')

In [10]:
times_relevant_columns = ['rank', 'name', 'location',  
       'scores_citations', 'scores_citations_rank', 'scores_industry_income',
       'scores_industry_income_rank', 'scores_international_outlook',
       'scores_international_outlook_rank', 'scores_overall',
       'scores_overall_rank', 'scores_research', 'scores_research_rank',
       'scores_teaching', 'scores_teaching_rank', 'stats_female_male_ratio',
       'stats_number_students', 'stats_pc_intl_students',
       'stats_student_staff_ratio']

top_unis_times = top_unis_times[times_relevant_columns]

top_unis_times['rank'] = top_unis_times['rank'].str.replace("=", "").apply(pd.to_numeric)

# Merging both ranking DataFrames
Strategy:
1. We checked how many of university names overlap (105).
2. We looked at those which were not matched and found out that many of them have an abbreviation at the end, so we deleted them using regex to find every string that contains parentheses, for example '(MIT)' or '(EPFL)'.
3. After that operation there were 119 matched universities.
4. Another differentiating thing was the word `The` at the beginning of universities' names (139 matched).
5. At the end we used difflib library and its `get_close_matches` function iteratively with different cutoff parameter to find best matches between not matched names. That part required more manual assigning.
6. We ended up with 160 matched universities.
7. Merging was made on respectively `name` and `title` columns.

In [11]:
matching_names_total_in_times = np.array([title in top_unis_times['name'].values for title in Total['title'].values])
matching_names_total_in_times

array([False,  True,  True, False,  True,  True, False,  True,  True,
       False, False, False,  True,  True, False,  True,  True,  True,
        True, False,  True,  True, False, False,  True, False, False,
       False,  True, False,  True,  True, False, False, False,  True,
        True,  True, False,  True, False, False, False,  True, False,
       False, False,  True,  True, False,  True, False,  True,  True,
        True, False, False,  True, False,  True,  True,  True, False,
        True,  True, False,  True, False,  True,  True, False,  True,
        True,  True, False, False, False,  True,  True, False,  True,
       False, False,  True, False, False,  True, False,  True, False,
        True,  True, False,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True, False, False,  True,
        True,  True, False,  True,  True, False,  True, False,  True,
        True,  True, False, False, False, False, False, False,  True,
        True, False,

This names were not matched between `Total` and `top_unis_times`

In [12]:
not_matched = Total[~matching_names_total_in_times].title.values
print(len(not_matched))

95


In [13]:
Total.loc[Total['title'] == 'UCL (University College London)', 'title'] = 'University College London'
top_unis_times.loc[top_unis_times['name'] == 'ETH Zurich – Swiss Federal Institute of Technology Zurich'
                   , 'name'] = 'ETH Zurich - Swiss Federal Institute of Technology'
Total.loc[Total['title'] == 'Ecole Polytechnique Fédérale de Lausanne (EPFL)', 'title'] = 'École Polytechnique Fédérale de Lausanne'

In [14]:
for not_match in not_matched:
    Total.loc[Total['title'] == not_match, 'title'] = re.sub(r"\((.*?)\)", "" , not_match).strip()

In [15]:
matching_names_total_in_times_2 = np.array([title in top_unis_times['name'].values for title in Total['title'].values])
print(np.count_nonzero(matching_names_total_in_times_2))

119


Plenty of names differ only by a `The` at the beginning of a `name`

In [16]:
similar = [(elem, difflib.get_close_matches(elem, top_unis_times['name'].values, cutoff = 0.7)[0]) for elem in Total[~matching_names_total_in_times_2].title.values if difflib.get_close_matches(elem, top_unis_times['name'].values, cutoff = 0.7) != []]
len(similar)

55

In [17]:
for not_match in Total[~matching_names_total_in_times_2].title.values:
    Total.loc[Total['title'] == not_match, 'title'] = re.sub(r"^(The)\s", "" , not_match)

In [18]:
matching_names_total_in_times_3 = np.array([title in top_unis_times['name'].values for title in Total['title'].values])
print(np.count_nonzero(matching_names_total_in_times_3))

139


In [19]:
similar2 = [(elem, difflib.get_close_matches(elem, top_unis_times['name'].values, cutoff = 0.4)[0]) 
            for elem in Total[~matching_names_total_in_times_3].title.values 
            if difflib.get_close_matches(elem, top_unis_times['name'].values, cutoff = 0.4) != []]

In [20]:
names = [("King's College London", 'King’s College London'),
         ('KAIST - Korea Advanced Institute of Science & Technology',
          'Korea Advanced Institute of Science and Technology (KAIST)'),
         ('Ecole Polytechnique', 'École Polytechnique'),
         ('Pohang University of Science And Technology',
          'Pohang University of Science and Technology'),
         ('University of North Carolina, Chapel Hill',
          'University of North Carolina at Chapel Hill'),
         ('Washington University in St. Louis', 'Washington University in St Louis'),
         ('KIT, Karlsruhe Institute of Technology',
          'Karlsruhe Institute of Technology'),
         ('Humboldt-Universität zu Berlin', 'Humboldt University of Berlin'),
         ('Freie Universitaet Berlin', 'Free University of Berlin'),
         ('Université de Montréal', 'University of Montreal'),
         ('Technische Universität Berlin', 'Technical University of Berlin'),
         ('Université catholique de Louvain', 'Université Catholique de Louvain'),
         ('Trinity College Dublin, The University of Dublin',
          'Trinity College Dublin'),
         ("Scuola Superiore Sant'Anna Pisa di Studi Universitari e di Perfezionamento",
          'Scuola Superiore Sant’Anna'),
         ('Universitat Autònoma de Barcelona', 'Autonomous University of Barcelona'),
         ('Eberhard Karls Universität Tübingen', 'University of Tübingen')]
         
for total_name, top_unis_name in names:        
    Total.loc[Total['title'] == total_name, 'title'] = top_unis_name

In [21]:
matching_names_total_in_times_4 = np.array([title in top_unis_times['name'].values for title in Total['title'].values])
print(np.count_nonzero(matching_names_total_in_times_4))

155


In [22]:
matching_names_times_in_total = np.array([title in Total['title'].values for title in top_unis_times['name'].values])
print(np.count_nonzero(matching_names_times_in_total))

155


In [23]:
similar3 = [(elem, difflib.get_close_matches(elem, top_unis_times[~matching_names_times_in_total].name.values, cutoff = 0.2)) 
            for elem in Total[~matching_names_total_in_times_4].title.values]

In [24]:
names2 = [('Ludwig-Maximilians-Universität München', 'LMU Munich'),
         ('Ruprecht-Karls-Universität Heidelberg', 'Heidelberg University'),
         ('Université Pierre et Marie Curie', 'Pierre and Marie Curie University'),
         ('Wageningen University', 'Wageningen University & Research'),
         ('Albert-Ludwigs-Universitaet Freiburg', 'University of Freiburg')]

for total_name, top_unis_name in names2:        
    Total.loc[Total['title'] == total_name, 'title'] = top_unis_name

In [25]:
matching_names_times_in_total = np.array([title in Total['title'].values for title in top_unis_times['name'].values])
print(np.count_nonzero(matching_names_times_in_total))

160


In [27]:
Full_ranking = Total.merge(top_unis_times, left_on='title', right_on='name', how='inner')
Full_ranking[['title', 'Rank TU', 'rank']]

,title,Rank TU,rank
0,Massachusetts Institute of Technology,1,5
1,Stanford University,2,3
2,Harvard University,3,6
3,California Institute of Technology,4,3
4,University of Cambridge,5,2
5,University of Oxford,6,1
6,University College London,7,16
7,Imperial College London,8,8
8,University of Chicago,9,9
9,ETH Zurich - Swiss Federal Institute of Techno...,10,10
